In [2]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import os

1. Phase별 데이터 전처리 + write

In [ ]:
base_path = '../data/open_track1/'
df = pd.read_csv(base_path + "train.csv")
df = df.sort_values(["game_episode", "time_seconds"]).reset_index(drop=True)

ph = 0
past_game_id = 0
past_team_id = 0
phase = []

for row in df.itertuples():
    if past_team_id != row.team_id:
        ph += 1
    
    if row.Index == 0:
        phase.append(str(row.game_id) + '_' + str(ph))
        continue
    if past_game_id != row.game_id:
        ph = 1

    

    phase.append(str(row.game_id) + '_' + str(ph))

    past_game_id = row.game_id
    past_team_id = row.team_id


df['phase'] = phase
# print(df.head())

# df.to_csv('phase_train.csv', index=False)


   game_id  period_id  episode_id  time_seconds  team_id  player_id  \
0   126283          1           1         0.667     2354     344559   
1   126283          1           1         3.667     2354     250036   
2   126283          1           1         4.968     2354     500145   
3   126283          1           1         8.200     2354     500145   
4   126283          1           1        11.633     2354     142106   

   action_id type_name result_name    start_x    start_y      end_x  \
0          0      Pass  Successful  52.418205  33.485444  31.322445   
1          2      Pass  Successful  32.013240  38.100808  37.371285   
2          4     Carry         NaN  37.371285  30.632980  38.391570   
3          5      Pass  Successful  38.391570  24.613144  34.573350   
4          7      Pass  Successful  34.578705   6.058256  21.274470   

       end_y  is_home game_episode     phase  
0  38.274752     True     126283_1  126283_1  
1  30.632980     True     126283_1  126283_1  
2  24

2. test 데이터 일단 경기별로 합치기

In [ ]:
tdp = '../../data/open_track1/test1'

filenames = os.listdir(tdp)

for game_id in filenames:
    game_data = os.listdir(tdp + '/' + game_id)
    if not os.path.exists('../../data/open_track1/test_totla/'+ game_id):
            os.mkdir('../../data/open_track1/test_total/'+ game_id)
    tot = []

    for file in game_data:
        file_path = os.path.join(tdp, game_id, file)
        tot.append(pd.read_csv(file_path))

    tot = pd.concat(tot, ignore_index=True)
    # tot.to_csv('../../data/open_track1/test_total/' + game_id + '/' + game_id + '_total.csv' , index=False)


3. test 데이터를 phase별로 나누고, total 파일에 phase column 추가

In [ ]:
tdp = '../data/test_total'

filenames = os.listdir(tdp)

for game_id in filenames:
    # define game-wise total data path
    game_path = tdp + '/' + game_id + '/' + game_id + '_total.csv'

    # read test file by game_id
    df = pd.read_csv(game_path)
    df = df.sort_values(["game_episode", "time_seconds"]).reset_index(drop=True)
    
    ph = 0
    past_game_id = 0
    past_team_id = 0
    phase = []

    for row in df.itertuples():
        if past_team_id != row.team_id:
            ph += 1
        
        if row.Index == 0:
            phase.append(str(row.game_id) + '_' + str(ph))
            continue
        if past_game_id != row.game_id:
            ph = 1

        

        phase.append(str(row.game_id) + '_' + str(ph))

        past_game_id = row.game_id
        past_team_id = row.team_id


    df['phase'] = phase
    # df.to_csv(tdp + '/' + game_id + '/' + game_id + '_total.csv' , index=False)


4. phase 폴더 만든 뒤, 게임 + phase별 .csv 파일 생성

In [ ]:
filenames = os.listdir(tdp)

for game_id in filenames:
    # define game-wise total data path
    game_path = tdp + '/' + game_id + '/' + game_id + '_total.csv'
    df = pd.read_csv(game_path)
    df = df.sort_values(["phase"]).reset_index(drop=True)
    for phase, row in df.groupby('phase'):
        g = row.reset_index(drop=True)
        # g.to_csv(tdp+'/'+game_id+'/phase/' + phase + '.csv')

5. phase별 데이터 접근하는 경로 적혀있는 phase_test.csv파일 생성

In [ ]:
records = []

for game_id in filenames:
    phase_files= os.listdir(tdp + '/' + game_id + '/phase')
    # print(len(phase_files))
    for phase_file in phase_files:
        data_path = tdp + '/' + game_id + '/phase/' + phase_file
        phase = phase_file[:-4]
        records.append({
            "game_id": game_id,
            "game_phase": phase,
            "path": data_path})

df = pd.DataFrame(records)
# df.to_csv('../data/open_track1/phase_test.csv', index=False)

6. 기존 episode별 test.csv에 phase 컬럼 추가해서 파일 생성

In [ ]:
tdp = '../data/test'
filenames = os.listdir(tdp)

list = []

for game_id in filenames:
    # define game-wise total data path
    game_path = '../data/test_total/' + game_id + '/' + game_id + '_total.csv'
    df = pd.read_csv(game_path)
    df = df.sort_values(["game_episode", "time_seconds"]).reset_index(drop=True)
    # if not os.path.exists('../data/test/'+ game_id):
    #         os.mkdir('../data/test/'+ game_id)

    for episode, row in sorted(df.groupby('game_episode'), key = lambda x: int(x[0])):
        g = row.reset_index(drop=True)
        # g.to_csv('../data/test/'+ game_id+'/' + episode + '.csv')


